In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

warnings.filterwarnings("ignore")

import inspect

from scipy.stats import norm, skew
from scipy.special import boxcox1p
## pre process 
from sklearn.preprocessing import LabelEncoder, StandardScaler

#Selection And Cross Validation 
from sklearn.model_selection import KFold, cross_val_score, train_test_split,GridSearchCV

#Modles
from sklearn.linear_model import Ridge
from sklearn.base import BaseEstimator, TransformerMixin, RegressorMixin, clone

# scoring 
from sklearn.metrics import mean_squared_error,r2_score

In [ ]:
train_df = pd.read_csv(r'../input/house-prices-advanced-regression-techniques/train.csv')
test_df = pd.read_csv(r'../input/house-prices-advanced-regression-techniques/test.csv')
df = train_df.copy()
target = np.log1p(train_df.SalePrice.copy())
target

In [ ]:
train_df.info()

In [ ]:
numerical = df.select_dtypes(include = ['int64','float64'])

In [ ]:
numerical

In [ ]:
null_num = numerical.isna().sum()
nulled = null_num[null_num>0].to_frame()
nulled

### LotFrontage distribution

In [ ]:
sns.distplot(numerical.LotFrontage);

### almost normal so I'll fill the missing data with the median of the data

### MasVnrArea distribution 

In [ ]:
sns.distplot(numerical.MasVnrArea);

### sence it's distrbuted arround 0 I'll fill the missing with 0

### GarageYrBlt distribution

In [ ]:
sns.distplot(numerical.GarageYrBlt);

In [ ]:
fig, ax = plt.subplots(figsize=(10, 6))
sns.heatmap(numerical.corr());

### sence it doesn't indicate anything I'll look on the rest of the Garage data

In [ ]:
df[['GarageYrBlt','GarageArea','GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','GarageYrBlt']]\
[df.GarageYrBlt.isnull()].isna().sum()

### all garage data are the same lost data in all its fields so we can impute them as a whole as no garage or 0 in numeric data

### the rest of categorical data with nulls

In [ ]:
categorical = df.select_dtypes(object)
categorical.columns

In [ ]:
cat = categorical.isnull().sum().sort_values()
null_cat = cat[cat > 0].to_frame()
null_cat

In [ ]:
sns.countplot(df.Electrical);

### sence most of the data is at SBrkr I'll fill the missing with the mode

### computing missing data

In [ ]:
df = pd.concat((train_df, test_df)).reset_index(drop=True)
df.drop(['SalePrice'], axis=1, inplace=True)
print("all_data size is : {}".format(df.shape))

### filling LotFrontage with median

In [ ]:
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(train_df.LotFrontage.median()))

In [ ]:
df["LotFrontage"].isna().sum()

### filling MasVnrArea with zeros

In [ ]:
df.MasVnrArea.fillna(0, inplace = True)

In [ ]:
df["MasVnrArea"].isna().sum()

### Filling GarageYrBlt with zeros

In [ ]:
df.GarageYrBlt.fillna(0,inplace = True)

### as we have finished imputing numerical data it's time to go for categorical data

### as Garage data are all related it will be filled with Not_existing

In [ ]:
cols = ['GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','GarageYrBlt']
for col in cols:
    df[col].fillna('Not_existing',inplace = True)

In [ ]:
df[['GarageYrBlt','GarageArea','GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','GarageYrBlt']]\
[df.GarageYrBlt.isnull()].isna().sum()

### most of data is at SBrkr so we can fill the data with mode

In [ ]:
co = ['MSZoning' , 'Utilities' , 'Exterior1st' , 'Exterior2nd' , 'KitchenQual' , 'Functional' , 'GarageCars' , 'GarageArea' ,'SaleType' ,'Electrical']
for c in co: 
    df[c].fillna(train_df[c].mode()[0],inplace = True)

### now by looking at data description I created a list of categorical data that includes columns with null data means the feature doesn't exist in the reality so it will be filled with not exiting 

In [ ]:
does_not_exist = ['MasVnrType','BsmtCond','BsmtQual','BsmtFinType1','BsmtExposure','BsmtFinType2',
                  'BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'
                  ,'FireplaceQu','Fence','Alley','MiscFeature','PoolQC']
for column in does_not_exist:
    df[column].fillna('Not_existing',inplace = True)

In [ ]:
df.isnull().sum().sort_values()

In [ ]:
nul = df.isna().sum()
d = nul[nul>0]
d

### now we have finished missing data imputing some feature engineering

### ordinal encoding for some features

In [ ]:
ord = ('FireplaceQu', 'BsmtQual', 'BsmtCond', 'GarageQual', 'GarageCond', 
        'ExterQual', 'ExterCond','HeatingQC', 'PoolQC', 'KitchenQual', 'BsmtFinType1', 
        'BsmtFinType2', 'Functional', 'Fence', 'BsmtExposure', 'GarageFinish', 'LandSlope',
        'LotShape', 'PavedDrive', 'Street', 'Alley', 'CentralAir')
for o in ord:
    lab = LabelEncoder()
    lab.fit(list(df[o].values))
    df[o] = lab.transform(list(df[o].values))

### checking for duplication

In [ ]:
pd.set_option('display.max_columns', None)
df[df.duplicated()]


### obviously this is not duplication in data due to year built is diffrent and other parms like Lot Area 1stFlrSF and 2ndFlrSF	

In [ ]:
df.drop(["Id"], axis=1, inplace=True)

In [ ]:
df = pd.get_dummies(df)

train = df[:train_df.shape[0]]
test = df[train_df.shape[0]:]
train.shape


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=42)

In [ ]:
params = {'alpha': [0.0001, 0.001, 0.01, 0.05, 0.1, 
                 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1.0, 2.0, 3.0, 
                 4.0, 5.0, 6.0, 7.0, 8.0, 9.0, 10.0, 20, 50, 100]
         }


ridge = Ridge()

folds = 5
model_cv = GridSearchCV(estimator = ridge, 
                        param_grid = params, 
                        scoring= 'neg_mean_squared_error', 
                        cv = folds, 
                        return_train_score=True,
                        verbose = 1)    

model_cv.fit(X_train, y_train) 

In [ ]:
ridge = model_cv.best_estimator_


In [ ]:
y_train_pred_ridge = ridge.predict(X_train)
print(r2_score(y_true=y_train, y_pred=y_train_pred_ridge))

In [ ]:
y_test_pred_ridge = ridge.predict(X_test)
print(r2_score(y_true=y_test, y_pred=y_test_pred_ridge))

In [ ]:
print ('RMSE Validation is: \n', mean_squared_error(y_test, y_test_pred_ridge))


In [ ]:
preds = np.exp(ridge.predict(test))

In [ ]:
predictions = pd.DataFrame({'Id': test_df['Id'] ,'SalePrice': preds })

In [ ]:
predictions.to_csv("preds.csv",index=False)

In [ ]:
predictions.SalePrice